# Image Denoising with SimpleITK

If SimpleITK is not yet installed in Your environment do so by executing:

In [1]:
## with conda
#!conda install -c simpleitk simpleitk

## with pip
!python -m pip install SimpleITK

Import SimpleITK as sitk, read in a 3D image and show it in Fiji/IJ (don't close the window) and get some stats using SITK.\
On W10 fiji needs to be installed [C:\Users\your_user_name\Fiji.app](https://simpleitk.readthedocs.io/en/master/faq.html#why-isn-t-fiji-or-imagej-found-by-the-show-function-runtimeerror-exception-thrown) for this to work.

In [1]:
import os
import SimpleITK as sitk

In [2]:
img = sitk.ReadImage("Bildserie_EM_Blender.mha")



In [3]:
sitk.Show(img, title='orig')

In [4]:
## there is no sitk.Statistics function, so use *ImageFilter() instance instead (http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/03_Image_Details.html)
statistics_image_filter = sitk.StatisticsImageFilter()
statistics_image_filter.Execute(img)
statistics_image_filter.GetMean()

136.26157494434136

You can get help on nearly every filter:

In [5]:
# help(sitk.StatisticsImageFilter)

### Exercise 1

Now run an isotropic 3D mean filter with a radis of 1 and inspect the result:

In [6]:
## read up on the expected parameters of the mean filter function
mean_image_filter = sitk.MeanImageFilter()

In [7]:
## execute the mean filter function
img_mean = mean_image_filter.Execute(img)

In [8]:
## inspect the result visually
sitk.Show(img_mean, title='Mean Image')

Nearly each ITK ImageFilter has a corresponding function in SITK for procedural execution (instead of object-oriented execution from filter classes)

In [9]:
## SITK filter function
help(sitk.GradientMagnitudeRecursiveGaussian)

Help on function GradientMagnitudeRecursiveGaussian in module SimpleITK.SimpleITK:

GradientMagnitudeRecursiveGaussian(image1, sigma=1.0, normalizeAcrossScale=False)
    GradientMagnitudeRecursiveGaussian(Image image1, double sigma=1.0, bool normalizeAcrossScale=False) -> Image



In [10]:
## SITK image filter class
help(sitk.GradientMagnitudeRecursiveGaussianImageFilter)

Help on class GradientMagnitudeRecursiveGaussianImageFilter in module SimpleITK.SimpleITK:

class GradientMagnitudeRecursiveGaussianImageFilter(ImageFilter)
 |  Computes the Magnitude of the Gradient of an image by convolution with
 |  the first derivative of a Gaussian.
 |  
 |  
 |  This filter is implemented using the recursive gaussian filters
 |  See:
 |   itk::simple::GradientMagnitudeRecursiveGaussian for the procedural interface
 |  
 |   itk::GradientMagnitudeRecursiveGaussianImageFilter for the Doxygen on the original ITK class.
 |  
 |  
 |  C++ includes: sitkGradientMagnitudeRecursiveGaussianImageFilter.h
 |  
 |  Method resolution order:
 |      GradientMagnitudeRecursiveGaussianImageFilter
 |      ImageFilter
 |      ProcessObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Execute(self, image1)
 |      Execute(GradientMagnitudeRecursiveGaussianImageFilter self, Image image1) -> Image
 |  
 |  GetName(self)
 |      GetName(GradientMagnitudeRecursiveGau

In [11]:
gm= sitk.GradientMagnitudeRecursiveGaussian(img, sigma=10.0)

In [12]:
sitk.Show(gm, title="gm+10.0")

Most image filters like GradientAnisotropicDiffusion (adg) can also be run object-oriented as in "pure" ITK which allows e.g. to introduce progress reports:

In [13]:
class MyCommand(sitk.Command):
    def __init__(self, po):
        # required
        super(MyCommand, self).__init__()
        self.processObject = po

    def Execute(self):
        print(f"{self.processObject.GetName()} Progress: {self.processObject.GetProgress():1.2f}")

adg= sitk.GradientAnisotropicDiffusionImageFilter()
adg.SetTimeStep(0.75)
adg.SetConductanceParameter(0.75)
adg.SetNumberOfIterations(10)

cmd=MyCommand(adg)
adg.AddCommand(sitk.sitkProgressEvent, cmd)
adg.AddCommand(sitk.sitkStartEvent, lambda: print("StartEvent"))
adg.AddCommand(sitk.sitkEndEvent, lambda: print("EndEvent"))

adg_img= adg.Execute(sitk.Cast(img, sitk.sitkFloat32))

StartEvent
GradientAnisotropicDiffusionImageFilter Progress: 0.00


GradientAnisotropicDiffusionImageFilter (0x7fcdd0bed6c0): Anisotropic diffusion unstable time step: 0.75
Stable time step for this image must be smaller than 0.3125



GradientAnisotropicDiffusionImageFilter Progress: 0.00
GradientAnisotropicDiffusionImageFilter Progress: 0.10


GradientAnisotropicDiffusionImageFilter (0x7fcdd0bed6c0): Anisotropic diffusion unstable time step: 0.75
Stable time step for this image must be smaller than 0.3125



GradientAnisotropicDiffusionImageFilter Progress: 0.10
GradientAnisotropicDiffusionImageFilter Progress: 0.20


...

GradientAnisotropicDiffusionImageFilter Progress: 0.70
GradientAnisotropicDiffusionImageFilter Progress: 0.80


GradientAnisotropicDiffusionImageFilter (0x7fcdd0bed6c0): Anisotropic diffusion unstable time step: 0.75
Stable time step for this image must be smaller than 0.3125



GradientAnisotropicDiffusionImageFilter Progress: 0.80
GradientAnisotropicDiffusionImageFilter Progress: 0.90


GradientAnisotropicDiffusionImageFilter (0x7fcdd0bed6c0): Anisotropic diffusion unstable time step: 0.75
Stable time step for this image must be smaller than 0.3125



GradientAnisotropicDiffusionImageFilter Progress: 0.90
GradientAnisotropicDiffusionImageFilter Progress: 1.00
EndEvent


Running GradientAnisotropicDiffusion (adg) procedural:\
(NB: the cell with the following commands is [disabled, cells of type "Raw NBConvert"](https://stackoverflow.com/questions/32444840/jupyter-how-to-comment-out-cells))

In [14]:
sitk.Show(adg_img, title="adg+0.75+0.75+10")

### Question 1

1. Do we need the gm result from above?
2. How is the gm result from above related to adg?
3. If you did not close the Fiji windows, you can use Fiji to do a quantitative comparison inside Fiji! \
   What is the value range the mean and the adg results differ by?

### Answer 1

1. Nein der gm Wert wird in der funktion nicht übergeben
2. Die Berechnungen des gm (Betrag der Gradienten) wird ebenfalls in der funktion adg (Ortsabhängiger Gradient) durchgeführt.
3. Der Differenzstack der beiden Stacks adg+0.75+0.75+10 und gm+10.0 besitzt ein Minimum von 28.094 und ein Maximum 246.810.

### Exercise 2

Double the `TimeStep` and half the `NumberOfIterations` and compare the result to the former "adg+0.75+0.75+10":

In [15]:
## double the TimeStep and half the NumberOfIterations of adg from above
adg2_img= sitk.GradientAnisotropicDiffusion(sitk.Cast(img, sitk.sitkFloat32), timeStep=1.5, conductanceParameter=0.75, numberOfIterations=5)

sitk.Show(adg2_img, title="adg+1.5+0.75+5")

GradientAnisotropicDiffusionImageFilter (0x7fcdd094f7f0): Anisotropic diffusion unstable time step: 1.5
Stable time step for this image must be smaller than 0.3125

GradientAnisotropicDiffusionImageFilter (0x7fcdd094f7f0): Anisotropic diffusion unstable time step: 1.5
Stable time step for this image must be smaller than 0.3125

GradientAnisotropicDiffusionImageFilter (0x7fcdd094f7f0): Anisotropic diffusion unstable time step: 1.5
Stable time step for this image must be smaller than 0.3125

GradientAnisotropicDiffusionImageFilter (0x7fcdd094f7f0): Anisotropic diffusion unstable time step: 1.5
Stable time step for this image must be smaller than 0.3125

GradientAnisotropicDiffusionImageFilter (0x7fcdd094f7f0): Anisotropic diffusion unstable time step: 1.5
Stable time step for this image must be smaller than 0.3125



### Question 2

1. If you did not close the Fiji windows, you can use Fiji to do a quantitative comparison inside Fiji!\
   What is the value range the two adg results differ by?

2. What happens for a too big `TimeStep` of about `6.0`? (NB: Check the docu!)

### Answer 2

1. Der Differenzstack der beiden Stacks adg+0.75+0.75+10 und adg+1.5+0.75+5 besitzt ein Minimum von -7.0899 und ein Maximum 8.1589.
2. Bei einem sehr gross gewählten `TimeStep`, wiird die verändung der "Wärmeverteilung" sehr groß und das bild somit pixelig.